In [105]:
import copy
import datetime
import hashlib

In [116]:
# Define block chain

class Chain():
    def __init__(self): 
        self.blocks = [self.get_genesis_block()]
    
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        else:
            return False
    
    def get_genesis_block(self): # genesis block
        return Block(
            index=0,
            timestamp=datetime.datetime.utcnow(),
            data='Genesis',
            previous_hash='arbitraray'
        )
    
    def add_block(self, data):
        self.blocks.append(Block(
            index=len(self.blocks),
            timestamp=datetime.datetime.utcnow(),
            data=data,
            previous_hash=self.blocks[len(self.blocks) - 1].hash
        ))
        
    def get_chain_size(self):
        return len(self.blocks) - 1
    
    def verify(self, verbose=True):
        flag = True
        for i in range(1, len(self.blocks)):
            if not self.blocks[i].verify():
                flag = False
                if verbose:
                    print(f'Wrong data type(s) at block {i}.')
            if self.blocks[i].index != i:
                flag = False
                if verbose:
                    print(f'Wrong block index at block {i}.')
            if self.blocks[i - 1].hash != self.blocks[i].previous_hash:
                flag = False
                if verbose:
                    print(f'Wrong block previous hash at block {i}.')
            if self.blocks[i].hash != self.blocks[i].hashing():
                flag = False
                if verbose:
                    print(f'Wrong hash at block {i}.')
            if self.blocks[i - 1].timestamp >= self.blocks[i].timestamp:
                flag = False
                if verbose:
                    print(f'Backdating at block {i}.')
        return flag
        
    def fork(self, head='latest'):
        if head in ['latest', 'whole', 'all']:
            return copy.deepcopy(self)
        else:
            c = copy.deepcopy(self)
            c.blocks = c.blocks[0 : head + 1]
            return c
    
    def get_root(self, chain_2):
        min_chain_size = min(self.get_chain_size(), chain_2.get_chain_size())
        for i in range(1, min_chain_size + 1):
            if self.blocks[i] != chain_2.blocks[i]:
                return self.fork(i - 1)
        return self.fork(min_chain_size)

In [117]:
class Block():
    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.hashing()
    
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.__dict__ == other.__dict__
        else:
            return False
    
    def hashing(self):
        key = hashlib.sha256()
        key.update(str(self.index).encode('utf-8'))
        key.update(str(self.timestamp).encode('utf-8'))
        key.update(str(self.data).encode('utf-8'))
        key.update(str(self.previous_hash).encode('utf-8'))
        return key.hexdigest()
    
    def verify(self):
        instances = [self.index, self.timestamp, self.previous_hash, self.hash]
        types = [int, datetime.datetime, str, str]
        if sum(map(lambda inst_, type_: isinstance(inst_, type_), instances, types)) == len(instances):
            return True
        else:
            return False

In [118]:
c = Chain()
for i in range(1, 21):
    c.add_block(f'This is block {i} of my first chain.')

In [119]:
print(c.blocks[3].timestamp)
print(c.blocks[3].data)
print(c.blocks[3].hash)
print(c.blocks[2].hash)
print(c.blocks[3].previous_hash)

2019-11-03 16:08:45.904016
This is block 3 of my first chain.
68a7d1c2921327dccd7017b76320e2c9e1dde625c0c0e646536ea0da30941d9f
6b35d43d27f3ee5363c817a413d2f90dc6e20234927c291952f48366b34ef29a
6b35d43d27f3ee5363c817a413d2f90dc6e20234927c291952f48366b34ef29a


In [120]:
print(c.get_chain_size())
print(c.verify())

20
True


In [121]:
c_forked = c.fork('latest')
print(c == c_forked)

True


In [122]:
c_forked.add_block('New block for forked chain!')
print(c.get_chain_size(), c_forked.get_chain_size())

20 21


In [123]:
c_forked = c.fork('latest')
c_forked.blocks[9].index = -9
c_forked.verify()

Wrong block index at block 9.
Wrong hash at block 9.


False

In [124]:
c_forked = c.fork('latest')
c_forked.blocks[16].timestamp = datetime.datetime(2000, 1, 1, 0, 0, 0, 0)
c_forked.verify()

Wrong hash at block 16.
Backdating at block 16.


False